<a href="https://colab.research.google.com/github/akfangus/MLproject/blob/main/%EC%8B%AC%EB%A6%AC%EC%84%B1%ED%96%A5_%EC%98%88%EC%B8%A1_%EC%A0%84%EC%B2%98%EB%A6%ACX_pycaret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import norm, skew


style.use('seaborn-talk')

krfont={'family':'Malgun Gothic', 'weight': 'bold', 'size' : 10}
matplotlib.rc('font', **krfont)
matplotlib.rcParams['axes.unicode_minus'] = False

%matplotlib inline


- goal : 심사 기준: AUC

## 데이터 변수 설명
- Q_A 질문 답
    - 1=Disagree, 2=Slightly disagree, 3=Neutral, 4=Slightly agree, 5=Agree.비동의 -> 동의순
- Q_E(a~t) : 질문을 답할 때까지의 시간
- age_group : 연령
- education : 교육 수준
    - 1=Less than high school, 2=High school, 3=University degree, 4=Graduate degree, 0=무응답/ 숫자가 클수록 좋ㅇ므
- engnat : 모국어가 영어
    - 1=yes, 2=no, 0=무응답
- familysize : 형제자매 수
- gender : 성별
    - Male, Female
- hand : 필기하는 손
    - 1=Right, 2=Left, 3=Both, 0=무응답
- married : 혼인 상태
    - 1=미혼, 2=현재 기혼, 3=이전 기혼, 0=기타
- race : 인종
    - Asian, Arab, Black, Indigenous Australian, Native American, White, Other
- religion : 종교
    - Agnostic, Atheist, Buddhist, Christian_Catholic, Christian_Mormon, Christian_Protestant, Christian_Other, Hindu, Jewish, Muslim, Sikh, Other
- tp__ : 순서형 변수 0에 가까울 수록 본인이 생각하는 정도가 강한 것
    - 7이 무응답이고 0~6이 정상데이터입니다 
- urban : 유년기의 거주 구역
    - 1=시골(시골), 2=교외, 3=도시(도시, 도시), 0=무 답변
- voted (타겟): 지난 해 국가 선거 투표 여부
    - 1= yes, 2= no

In [ ]:
train = pd.read_csv('open data/train.csv')
test = pd.read_csv('open data/test_x.csv')
submission = pd.read_csv('open data/sample_submission.csv')

In [ ]:
train.columns

Index(['index', 'QaA', 'QaE', 'QbA', 'QbE', 'QcA', 'QcE', 'QdA', 'QdE', 'QeA',
       'QeE', 'QfA', 'QfE', 'QgA', 'QgE', 'QhA', 'QhE', 'QiA', 'QiE', 'QjA',
       'QjE', 'QkA', 'QkE', 'QlA', 'QlE', 'QmA', 'QmE', 'QnA', 'QnE', 'QoA',
       'QoE', 'QpA', 'QpE', 'QqA', 'QqE', 'QrA', 'QrE', 'QsA', 'QsE', 'QtA',
       'QtE', 'age_group', 'education', 'engnat', 'familysize', 'gender',
       'hand', 'married', 'race', 'religion', 'tp01', 'tp02', 'tp03', 'tp04',
       'tp05', 'tp06', 'tp07', 'tp08', 'tp09', 'tp10', 'urban', 'voted',
       'wf_01', 'wf_02', 'wf_03', 'wr_01', 'wr_02', 'wr_03', 'wr_04', 'wr_05',
       'wr_06', 'wr_07', 'wr_08', 'wr_09', 'wr_10', 'wr_11', 'wr_12', 'wr_13'],
      dtype='object')

In [ ]:
train[['tp01', 'tp02', 'tp03', 'tp04',
       'tp05', 'tp06', 'tp07', 'tp08', 'tp09', 'tp10']]

,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10
0,2,2,2,1,2,1,7,4,4,3
1,1,1,0,0,1,2,3,4,0,4
2,2,3,1,5,3,4,2,6,1,3
3,2,4,1,1,1,3,1,3,1,3
4,1,1,1,6,0,2,0,6,2,6
...,...,...,...,...,...,...,...,...,...,...
45527,1,2,6,1,2,6,3,0,6,4
45528,2,2,2,3,1,3,3,4,1,5
45529,4,4,2,4,0,1,0,2,1,6
45530,1,2,0,4,0,1,3,2,2,4


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45532 entries, 0 to 45531
Data columns (total 78 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       45532 non-null  int64  
 1   QaA         45532 non-null  float64
 2   QaE         45532 non-null  int64  
 3   QbA         45532 non-null  float64
 4   QbE         45532 non-null  int64  
 5   QcA         45532 non-null  float64
 6   QcE         45532 non-null  int64  
 7   QdA         45532 non-null  float64
 8   QdE         45532 non-null  int64  
 9   QeA         45532 non-null  float64
 10  QeE         45532 non-null  int64  
 11  QfA         45532 non-null  float64
 12  QfE         45532 non-null  int64  
 13  QgA         45532 non-null  float64
 14  QgE         45532 non-null  int64  
 15  QhA         45532 non-null  float64
 16  QhE         45532 non-null  int64  
 17  QiA         45532 non-null  float64
 18  QiE         45532 non-null  int64  
 19  QjA         45532 non-nul

In [ ]:
train_df = train.copy()
test_df = test.copy()

In [ ]:
category = train_df.select_dtypes(exclude=['int64','float64'])

numeric = train_df.select_dtypes(include=['int64','float64'])

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
one = OneHotEncoder(sparse=False)

In [ ]:
one.fit(category)

OneHotEncoder(sparse=False)

In [ ]:
one.transform(category)

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [ ]:
cate_XX = pd.DataFrame(one.transform(category), columns= one.get_feature_names())

X = pd.concat([numeric, cate_XX],axis=1, join='inner' )

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [ ]:
yy = X['voted']

In [ ]:
X.drop(['voted','index'], axis=1, inplace=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(XX,y, test_size=0.3, random_state=42)

In [ ]:
logit = LogisticRegression()

In [ ]:
logit.fit(X_train,y_train)

C:\Users\sh.lee\anaconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
logit.score(X_train,y_train)

0.5482868975903614

In [ ]:
rf = RandomForestClassifier()
GBA = GradientBoostingClassifier()

In [ ]:
rf.fit(X_train, y_train)
rf.score(X_train, y_train)

1.0

In [ ]:
GBA.fit(X_train, y_train)
GBA.score(X_train, y_train)

0.7078313253012049

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
rf.score(X_test, y_test)

0.6866764275256223

In [ ]:
GBA.score(X_test, y_test)

0.6965592972181552

In [ ]:
pred = GBA.predict(X_test)

In [ ]:
accuracy_score(y_test, pred)

0.6965592972181552

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb = XGBClassifier()

In [ ]:
xgb.fit(X_train,y_train)

C:\Users\sh.lee\anaconda3\envs\test\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:24:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=16, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
xgb.score(X_train,y_train)

In [ ]:
xgb.score(X_test, y_test)

0.6727672035139092

In [ ]:
pred = xgb.predict(X_test)

In [ ]:
accuracy_score(y_test, pred)

0.6727672035139092

## test

In [ ]:
category = test_df.select_dtypes(exclude=['int64','float64'])

numeric = test_df.select_dtypes(include=['int64','float64'])

In [ ]:
one.transform(category)

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [ ]:
cate_XX = pd.DataFrame(one.transform(category), columns= one.get_feature_names())

XX = pd.concat([numeric, cate_XX],axis=1, join='inner' )

In [ ]:
XX.drop('index', axis=1, inplace=True)

In [ ]:
X.shape, XX.shape

((45532, 100), (11383, 100))

In [ ]:
GBA.fit(X, yy)

GradientBoostingClassifier()

In [ ]:
pred_y = GBA.predict(XX)

In [ ]:
submission['voted']=pred_y

In [ ]:
submission

,index,voted
0,0,2
1,1,2
2,2,1
3,3,1
4,4,2
...,...,...
11378,11378,1
11379,11379,2
11380,11380,1
11381,11381,1


In [ ]:
submission.to_csv('sample_submission1.csv', index=False)

In [ ]:
!pip install pycaret

^C


In [ ]:
train

,index,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,...,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,0,3.0,363,4.0,1370,5.0,997,1.0,1024,2.0,...,0,1,0,1,1,0,1,0,1,1
1,1,5.0,647,5.0,1313,3.0,3387,5.0,2969,1.0,...,1,1,0,1,1,0,1,0,1,1
2,2,4.0,1623,1.0,1480,1.0,1021,4.0,3374,5.0,...,1,1,0,1,1,1,1,0,1,1
3,3,3.0,504,3.0,2311,4.0,992,3.0,3245,1.0,...,0,0,0,0,1,0,1,0,1,1
4,4,1.0,927,1.0,707,5.0,556,2.0,1062,1.0,...,1,1,1,1,1,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45527,45527,2.0,1050,5.0,619,4.0,328,1.0,285,1.0,...,1,1,0,1,1,1,1,0,1,1
45528,45528,2.0,581,3.0,1353,4.0,1164,1.0,798,3.0,...,1,1,0,1,1,1,1,0,1,1
45529,45529,4.0,593,1.0,857,1.0,1047,4.0,1515,5.0,...,1,1,0,1,1,0,1,0,1,1
45530,45530,1.0,747,3.0,1331,4.0,892,2.0,1281,1.0,...,1,1,0,1,1,1,1,0,1,1


In [ ]:
import pycaret
from pycaret.classification import *

In [ ]:
import xgboost
import catboost

## 실험 환경 구축 (Setup the environment)
- PyCaret에서는 모델 학습 전 실험 환경을 구축 해주어야 합니다. setup 함수를 통해 환경을 구축할 수 있습니다.
- setup 단계에서는 PyCaret이 자동으로 컬럼 형태를 인식합니다. 그 후 사용자에게 제대로 인식되었는지 확인을 받게 됩니다. 그 때 enter를 눌러주시면 됩니다.
- 또한 주어진 데이터의 얼마를 사용하여 train / validation을 구축할지 묻게 되는데, 전체 데이터를 사용하고 싶다면 enter 눌러주시면 됩니다.

In [ ]:
clf = setup(data=train, target='voted')

,Description,Value
0,session_id,2414
1,Target,voted
2,Target Type,Binary
3,Label Encoded,"1: 0, 2: 1"
4,Original Data,"(45532, 78)"
5,Missing Values,False
6,Numeric Features,42
7,Categorical Features,35
8,Ordinal Features,False
9,High Cardinality Features,False


## 모델 학습 및 비교 (Train models and compare)
- 환경 구축을 했으니 PyCaret에서 제공하는 기본 모델에 대해 학습하고 비교해보겠습니다.
- compared_models 함수를 통해 15개의 기본 모델을 학습하고 성능을 비교할 수 있습니다.
- AUC 기준으로 성능이 가장 좋은 3개의 모델을 추려내어 저장해보겠습니다. 본 대회 평가지표가 AUC이기 때문에 AUC 기준으로 모델을 선정합니다.

In [ ]:
best_3 = compare_models(sort = 'AUC', n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.6946,0.7670,0.6414,0.7653,0.6979,0.3940,0.4003,4.0590
lightgbm,Light Gradient Boosting Machine,0.6939,0.7654,0.6512,0.7582,0.7006,0.3912,0.3960,0.3040
lda,Linear Discriminant Analysis,0.6943,0.7636,0.6667,0.7497,0.7057,0.3899,0.3929,0.8890
et,Extra Trees Classifier,0.6928,0.7620,0.6508,0.7566,0.6997,0.3888,0.3935,2.3120
ada,Ada Boost Classifier,0.6931,0.7590,0.6607,0.7513,0.7030,0.3881,0.3916,0.8650
rf,Random Forest Classifier,0.6901,0.7564,0.6452,0.7557,0.6960,0.3840,0.3891,1.7120
dt,Decision Tree Classifier,0.6084,0.6046,0.6425,0.6444,0.6434,0.2091,0.2092,0.3990
lr,Logistic Regression,0.5516,0.5869,0.9907,0.5514,0.7085,0.0062,0.0219,2.1460
nb,Naive Bayes,0.4808,0.5275,0.3136,0.5402,0.2526,-0.0012,-0.0025,0.0870
knn,K Neighbors Classifier,0.5151,0.5103,0.5642,0.5586,0.5614,0.0194,0.0194,8.4120


## 모델 앙상블 (Model Ensemble)
- 학습된 3개의 모델을 앙상블 시키도록 하겠습니다. 본 대회는 score 최적화를 위해 확률 값을 예측해야 하므로 soft vote ensemble을 진행하겠습니다.

In [ ]:
blended = blend_models(estimator_list = best_3, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6960,0.7718,0.6640,0.7539,0.7061,0.3939,0.3973
1,0.7015,0.7713,0.6557,0.7676,0.7073,0.4066,0.4118
2,0.6992,0.7661,0.6571,0.7631,0.7061,0.4017,0.4064
3,0.6909,0.7666,0.6593,0.7486,0.7012,0.3838,0.3871
4,0.6971,0.7661,0.6586,0.7588,0.7051,0.3968,0.4010
Mean,0.6969,0.7684,0.6589,0.7584,0.7052,0.3965,0.4007
SD,0.0036,0.0026,0.0028,0.0067,0.0021,0.0077,0.0084


In [ ]:
blended

VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.1,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                             

## 모델 예측 (Prediction)
- 구축된 앙상블 모델을 통해 예측을 해보겠습니다.
- setup 환경에 이미 hold-out set이 존재하므로 해당 데이터에 대해 예측을 하여 모델 성능을 확인하겠습니다.

In [ ]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.6902,0.7610,0.6494,0.7439,0.6934,0.3832,0.3868


## 전체 데이터에 대한 재학습 (Re-training the model on whole data)
- 현재까지 실험은 주어진 train 데이터를 다시 한 번 train / validation으로 나눠서 실험을 한 것이므로, 전체 train 데이터에 학습되어 있지 않습니다.
- 최적의 성능을 위해 전체 데이터에 학습을 시켜주도록 하겠습니다.

In [ ]:
final_model = finalize_model(blended)

## 대회용 test set에 대한 예측 (Predicting on test set for the competition)
- predict_model 함수를 통해 재학습된 모델을 대회용 test set에 대해 예측해보겠습니다.
- We will now use the re-trained model on the test set for the competition

In [ ]:
predictions = predict_model(final_model, data = test)

In [ ]:
predictions

,index,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,...,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13,Label,Score
0,0,3.0,736,2.0,2941,3.0,4621,1.0,4857,2.0,...,0,0,1,0,1,0,1,1,2,0.6926
1,1,3.0,514,2.0,1952,3.0,1552,3.0,821,4.0,...,0,0,0,0,0,0,0,0,2,0.8639
2,2,3.0,500,2.0,2507,4.0,480,2.0,614,2.0,...,0,1,1,0,1,0,1,1,1,0.5670
3,3,1.0,669,1.0,1050,5.0,1435,2.0,2252,5.0,...,1,1,1,1,1,1,1,1,1,0.8170
4,4,2.0,499,1.0,1243,5.0,845,2.0,1666,2.0,...,0,1,1,0,1,1,1,1,2,0.7744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11378,11378,5.0,427,5.0,1066,5.0,588,1.0,560,2.0,...,0,1,1,0,1,0,1,1,1,0.6343
11379,11379,1.0,314,5.0,554,5.0,230,1.0,956,2.0,...,1,1,1,1,1,1,1,1,2,0.8645
11380,11380,1.0,627,2.0,799,1.0,739,2.0,1123,1.0,...,0,1,1,0,1,0,1,1,1,0.7807
11381,11381,2.0,539,1.0,2090,2.0,4642,1.0,673,2.0,...,0,1,1,0,1,1,1,0,1,0.6479


In [ ]:
final_model

VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.1,
                                                         loss='deviance',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                             

In [ ]:
test

,index,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,...,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,0,3.0,736,2.0,2941,3.0,4621,1.0,4857,2.0,...,0,1,0,0,1,0,1,0,1,1
1,1,3.0,514,2.0,1952,3.0,1552,3.0,821,4.0,...,0,0,0,0,0,0,0,0,0,0
2,2,3.0,500,2.0,2507,4.0,480,2.0,614,2.0,...,0,1,0,1,1,0,1,0,1,1
3,3,1.0,669,1.0,1050,5.0,1435,2.0,2252,5.0,...,1,1,1,1,1,1,1,1,1,1
4,4,2.0,499,1.0,1243,5.0,845,2.0,1666,2.0,...,1,1,0,1,1,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11378,11378,5.0,427,5.0,1066,5.0,588,1.0,560,2.0,...,1,1,0,1,1,0,1,0,1,1
11379,11379,1.0,314,5.0,554,5.0,230,1.0,956,2.0,...,1,1,1,1,1,1,1,1,1,1
11380,11380,1.0,627,2.0,799,1.0,739,2.0,1123,1.0,...,1,1,0,1,1,0,1,0,1,1
11381,11381,2.0,539,1.0,2090,2.0,4642,1.0,673,2.0,...,1,0,0,1,1,0,1,1,1,0


In [ ]:
submission['voted'] = predictions['Score']

In [ ]:
submission.to_csv('result.csv',index=False)